In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
import sqlite3
from sqlite3 import Error

In [24]:
conn = sqlite3.connect('wine_data.sqlite')
c = conn.cursor()


In [25]:
df = pd.read_sql('Select * from wine_data where price < 100 and price in (select price from wine_data group by price having count(price) > 10)', conn)
df.head(1)

,level_0,index,country,description,rating,price,province,title,variety,winery,color,countryID,varietyID,colorID,provinceID,wineryID
0,0,0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red,0,0,0,0,0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96640 entries, 0 to 96639
Data columns (total 16 columns):
level_0        96640 non-null int64
index          96640 non-null int64
country        96640 non-null object
description    96640 non-null object
rating         96640 non-null int64
price          96640 non-null float64
province       96640 non-null object
title          96640 non-null object
variety        96640 non-null object
winery         96640 non-null object
color          96640 non-null object
countryID      96640 non-null int64
varietyID      96640 non-null int64
colorID        96640 non-null int64
provinceID     96640 non-null int64
wineryID       96640 non-null int64
dtypes: float64(1), int64(8), object(7)
memory usage: 11.8+ MB


In [27]:
features_df = df.copy()
del features_df['price']
del features_df['index']
del features_df['description']
del features_df['title']
del features_df['province']
del features_df['winery']
del features_df['country']
del features_df['color']
del features_df['variety']
del features_df['provinceID']
del features_df['wineryID']
del features_df['level_0']
features_df.head(1)

,rating,countryID,varietyID,colorID
0,87,0,0,0


In [28]:
df.head(1)

,level_0,index,country,description,rating,price,province,title,variety,winery,color,countryID,varietyID,colorID,provinceID,wineryID
0,0,0,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,red,0,0,0,0,0


In [65]:
#create X and y arrays from the dataset using the .values command

X = features_df.values
y = df['price'].values

In [66]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.25, shuffle = True)

In [67]:
model = ensemble.GradientBoostingRegressor(
    n_estimators = 300, #how many decision trees to build
    learning_rate = 0.9, #controls rate at which additional decision trees influes overall prediction
    max_depth = 6, 
    min_samples_split = 21,
    min_samples_leaf = 19, 
    max_features = 0.9,
    loss = 'huber'
)

In [68]:
model.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.9, loss='huber', max_depth=6,
             max_features=0.9, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=19, min_samples_split=21,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [69]:
joblib.dump(model, 'xbgWinePrice.pkl')

['xbgWinePrice.pkl']

In [70]:
#Evaluate Results accuracy using Mean Absolute Error

mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training set mean absolute error: %.2f" % mse)

mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test set mean absolute error %2.f" %mse)

Training set mean absolute error: 9.19
Test set mean absolute error  9


In [71]:
y_pred = model.predict(X_test[:5]).round()
y_pred

array([37., 19., 26., 36., 27.])

In [72]:
y_test[:5]

array([30., 16., 19., 25., 35.])

In [73]:
model.score(X_test, y_test)

0.5069690965916411

In [74]:
job_model = joblib.load("xbgWinePrice.pkl")

In [75]:
job_model.predict(X_test[:5]).round()

array([37., 19., 26., 36., 27.])

In [81]:
def Test_model(rating, country, variety, color):
    
    dfco = df.loc[(df.country == country)][:1]
    dfv = df.loc[(df.variety ==variety)][:1]
    dfc = df.loc[(df.color == color)][:1]


    
    a = list(dfco['countryID'])
    b = list(dfv['varietyID'])
    c = list(dfv['colorID'])   


    dft = pd.DataFrame({'rating':rating,
                        "countryID":a,
                        "varietyID":b,
                        "colorID":c,})
    
    return model.predict(dft).round()
                      

In [82]:
Test_model(87, 'Portugal','Portuguese Red','red')

array([15.])

In [78]:
g = list(df['price'][:1])
print("Actual Price: ")
g

Actual Price: 


[15.0]